In [1]:
import pandas as pd
import pickle
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
from tqdm import tqdm

pd.set_option('max_rows',500,'max_colwidth',1000)

In [11]:
run_filename = '/data/Bodenreider_UMLS_DL/Interns/Bernal/few-shot-bioIE/outputs/rw-uva-2020AB-one-way/plms/2/cv.ybzm376a/enukuzv1/'

In [3]:
from UMLS import UMLS

umls = UMLS()

Loading Raw MRCONSO Lines


15229292it [00:52, 291863.20it/s]                                                                                                      


Creating mappings between concept IDs for easy access.


 38%|████████████████████████████████                                                    | 5806899/15229292 [01:34<14:50, 10582.51it/s]

('A1956370', 'C0893808', 'DMB-ceramide', 'M0359226', 'MSH', 'S', 'PF', 'ENG', 'L1760645')


 54%|██████████████████████████████████████████████▎                                      | 8286593/15229292 [02:08<15:36, 7415.69it/s]

('A9306521', 'C1634633', 'Capnocytophaga sp. AHN9821', '327651', 'NCBI', 'P', 'PF', 'ENG', 'L5755486')


 73%|█████████████████████████████████████████████████████████████▍                      | 11141364/15229292 [02:53<17:30, 3892.74it/s]

('A19199071', 'C3019765', 'Diptera sp. BOLD:AAI2104', '821727', 'NCBI', 'P', 'PF', 'ENG', 'L9764096')


100%|███████████████████████████████████████████████████████████████████████████████████| 15229292/15229292 [04:04<00:00, 62317.34it/s]


In [12]:
with open(run_filename + 'dev.metrics.3_results.json','r') as f:
    lines = f.readlines()

In [13]:
train_set = pd.read_csv('/data/Bodenreider_UMLS_DL/Interns/Bernal/few-shot-bioIE/data/rw-uva-2020AB-one-way/training_subsets/2/subset.csv',sep='\t')
train_set.groupby('label').count()

,Unnamed: 0,id,sents
label,,,
0,50000,50000,50000
1,50000,50000,50000


In [14]:
full_dev_set = pd.read_csv('/data/Bodenreider_UMLS_DL/Interns/Bernal/few-shot-bioIE/data/rw-uva-2020AB-one-way/dev.tsv',sep='\t')
full_dev_set.groupby('labels').count()

,Unnamed: 0,sents
labels,,
0,138016,138016
1,8239,8239


In [15]:
lines

['{"dev_f1": 0.4670326163273122, "dev_precision": 0.3168218229053908, "dev_recall": 0.8880932151960189, "dev_class_f1s": [0.9360584433485465, 0.4670326163273122]}']

In [16]:
with open(run_filename+'dev.tokens.3.txt','r') as f:
    dev_tokens = f.readlines()
    
with open(run_filename+'dev.labels.3.txt','r') as f:
    dev_labels = f.readlines()
    
with open(run_filename+'dev.predictions.3.txt','r') as f:
    dev_preds = f.readlines()

In [17]:
df = pd.DataFrame([t.strip() for t in dev_tokens], columns=['syn_pair'])

In [18]:
df['label'] = [int(l.strip()) for l in dev_labels]
df['pred'] = [int(l.strip()) for l in dev_preds]

In [19]:
df.groupby('label').count()

,syn_pair,pred
label,,
0,138016,138016
1,8239,8239


In [20]:
df.groupby('pred').count()

,syn_pair,label
pred,,
0,123160,123160
1,23095,23095


In [21]:
queries = []
candidates = []

for i in df.syn_pair:
    query, candidate = i.split(' [SEP] ')
    queries.append(query)
    candidates.append(candidate)

In [22]:
df['queries'] = queries
df['candidates'] = candidates

In [23]:
df['correct'] = df['label'] == df['pred']

In [24]:
query_auis = []
num_auis = []
sem_groups = []

for q in df.queries:
    
    auis = umls.str2aui[q]
    aui = auis[0]
    query_auis.append(aui)
    num_auis.append(len(auis))
    sem_groups.append(umls.aui2sg[aui])
    
df['query_auis'] = query_auis
df['num_auis'] = num_auis
df['sem_group'] = sem_groups

In [25]:
df['preferred_name'] = [umls.cui2preferred[umls.aui2cui[aui]] for aui in df.query_auis]

In [26]:
df_by_sem_group = df.groupby('sem_group').agg(['mean','count'])

recall_by_sg = []
precision_by_sq = []
f1_by_sq = []

for i,g in df.groupby('sem_group'):
    p, r, f, s = precision_recall_fscore_support(y_pred=g.pred, y_true=g.label,
                                                         labels=[1],
                                                         average='micro')
    recall_by_sg.append(r)
    precision_by_sq.append(p)
    f1_by_sq.append(f)
    
df_by_sem_group['recall'] = recall_by_sg
df_by_sem_group['precision'] = precision_by_sq
df_by_sem_group['f1'] = f1_by_sq

df_by_sem_group.sort_values(('label','count'),ascending=False)

label             pred          correct  \
                                 mean  count      mean  count      mean   
sem_group                                                                 
Living Beings                0.006821  59518  0.019221  59518  0.982123   
Chemicals & Drugs            0.129692  39239  0.319274  39239  0.789572   
Genes & Molecular Sequences  0.071789  19641  0.274935  19641  0.780765   
Disorders                    0.072246  13025  0.162380  13025  0.889290   
Procedures                   0.019295   5960  0.083221   5960  0.928356   
Physiology                   0.008427   4272  0.054073   4272  0.949204   
Concepts & Ideas             0.034125   2022  0.140950   2022  0.879327   
Anatomy                      0.075815   1319  0.435936   1319  0.630781   
Devices                      0.022222    405  0.034568    405  0.987654   
Activities & Behaviors       0.024306    288  0.239583    288  0.756944   
Objects                      0.095070    284  0.207746    284  0.887324   
Phenomena                    0.106383    282  0.634752    282  0.471631   

                                    num_auis           recall precision  \
                             count      mean  count                       
sem_group                                                                 
Living Beings                59518  1.084327  59518  0.598522  0.212413   
Chemicals & Drugs            39239  1.452789  39239  0.919631  0.373563   
Genes & Molecular Sequences  19641  1.277481  19641  0.887943  0.231852   
Disorders                    13025  2.252361  13025  0.857598  0.381560   
Procedures                    5960  1.309060   5960  0.800000  0.185484   
Physiology                    4272  1.046816   4272  0.694444  0.108225   
Concepts & Ideas              2022  1.412463   2022  0.797101  0.192982   
Anatomy                       1319  2.208491   1319  0.940000  0.163478   
Devices                        405  1.000000    405  1.000000  0.642857   
Activities & Behaviors         288  3.138889    288  0.428571  0.043478   
Objects                        284  3.334507    284  1.000000  0.457627   
Phenomena                      282  1.000000    282  1.000000  0.167598   

                                   f1  
                                       
sem_group                              
Living Beings                0.313548  
Chemicals & Drugs            0.531305  
Genes & Molecular Sequences  0.367695  
Disorders                    0.528141  
Procedures                   0.301146  
Physiology                   0.187266  
Concepts & Ideas             0.310734  
Anatomy                      0.278519  
Devices                      0.782609  
Activities & Behaviors       0.078947  
Objects                      0.627907  
Phenomena                    0.287081

In [27]:
df_by_query_concept = df.groupby('queries').agg(['mean'])

recall_by_query = []
precision_by_q = []
f1_by_q = []

for i,g in df.groupby('queries'):
    p, r, f, s = precision_recall_fscore_support(y_pred=g.pred, y_true=g.label,
                                                         labels=[1],
                                                         average='micro')
    recall_by_query.append(r)
    precision_by_q.append(p)
    f1_by_q.append(f)

/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_s

/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_s

/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_s

/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_s

/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gsfs8/users/jimenezgutierb2/conda/envs/rw_uva1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

In [28]:
df_by_query_concept['recall'] = recall_by_query
df_by_query_concept['precision'] = precision_by_q
df_by_query_concept['f1'] = f1_by_q

In [29]:
df_by_query_concept[df_by_query_concept[('label','mean')] == 0.0].mean()

label      mean    0.000000
pred       mean    0.027502
correct    mean    0.972498
num_auis   mean    1.066429
recall             0.000000
precision          0.000000
f1                 0.000000
dtype: float64

In [30]:
df_by_query_concept.mean()

label      mean    0.053492
pred       mean    0.152973
correct    mean    0.888193
num_auis   mean    1.374488
recall             0.358383
precision          0.177215
f1                 0.213294
dtype: float64

In [31]:
df_by_query_concept[df_by_query_concept[('label','mean')] > 0.0].mean()

label      mean    0.125872
pred       mean    0.322751
correct    mean    0.774117
num_auis   mean    1.791332
recall             0.843321
precision          0.417009
f1                 0.501908
dtype: float64

In [32]:
query_auis = []
num_auis = []
sem_groups = []

for q in df_by_query_concept.index:
    
    auis = umls.str2aui[q]
    aui = auis[0]
    query_auis.append(auis)
    num_auis.append(len(auis))
    sem_groups.append(umls.aui2sg[aui])
    
df_by_query_concept['query_auis'] = query_auis
df_by_query_concept['num_auis'] = num_auis
df_by_query_concept['sem_group'] = sem_groups

In [33]:
df_by_query_concept_by_sem_group = df_by_query_concept.groupby('sem_group').agg(['count','mean'])

In [34]:
df_by_query_concept_by_sem_group.sort_values(('label','mean','count'),ascending=False)

label            pred           correct            \
                             mean            mean              mean             
                            count      mean count      mean   count      mean   
sem_group                                                                       
Living Beings                 598  0.007656   598  0.020658     598  0.981002   
Chemicals & Drugs             382  0.119362   382  0.296914     382  0.802888   
Genes & Molecular Sequences   203  0.072590   203  0.277011     203  0.779272   
Disorders                     132  0.069755   132  0.164973     132  0.885837   
Procedures                     60  0.020109    60  0.085531      60  0.927125   
Physiology                     43  0.008564    43  0.054495      43  0.948876   
Concepts & Ideas               21  0.038991    21  0.151986      21  0.871701   
Anatomy                        14  0.079997    14  0.449990      14  0.621586   
Devices                         4  0.021429     4  0.033810       4  0.987619   
Activities & Behaviors          3  0.026515     3  0.256364       3  0.739848   
Objects                         3  0.097247     3  0.212415       3  0.884832   
Phenomena                       3  0.109890     3  0.655348       3  0.454542   

                            num_auis           recall           precision  \
                                mean                                        
                               count      mean  count      mean     count   
sem_group                                                                   
Living Beings                    598  1.093645    598  0.067529       598   
Chemicals & Drugs                382  1.497382    382  0.541962       382   
Genes & Molecular Sequences      203  1.275862    203  0.800921       203   
Disorders                        132  2.378788    132  0.475129       132   
Procedures                        60  1.333333     60  0.365556        60   
Physiology                        43  1.046512     43  0.255814        43   
Concepts & Ideas                  21  1.428571     21  0.228906        21   
Anatomy                           14  2.428571     14  0.678877        14   
Devices                            4  1.000000      4  0.250000         4   
Activities & Behaviors             3  3.333333      3  0.142857         3   
Objects                            3  3.333333      3  0.666667         3   
Phenomena                          3  1.000000      3  0.666667         3   

                                         f1            
                                                       
                                 mean count      mean  
sem_group                                              
Living Beings                0.046714   598  0.046919  
Chemicals & Drugs            0.286270   382  0.348961  
Genes & Molecular Sequences  0.322257   203  0.416096  
Disorders                    0.269887   132  0.301602  
Procedures                   0.195253    60  0.223800  
Physiology                   0.075631    43  0.108604  
Concepts & Ideas             0.071542    21  0.098080  
Anatomy                      0.197011    14  0.285057  
Devices                      0.225000     4  0.236842  
Activities & Behaviors       0.017241     3  0.030769  
Objects                      0.315603     3  0.428105  
Phenomena                    0.112374     3  0.192325

In [35]:
precision_recall_fscore_support(y_pred=df.pred, y_true=df.label,
                                                         labels=[1],
                                                         average='micro')

(0.3168218229053908, 0.8880932151960189, 0.4670326163273122, None)

In [36]:
#False Negatives
fns = df[(df['label'] != df['pred']) & (df['label'] == 1)]
print(len(fns))
fns[:10]

922


,syn_pair,label,pred,queries,candidates,correct,query_auis,num_auis,sem_group,preferred_name
123,"Dab, mitogen-responsive phosphoprotein, homolog 2 (Drosophila) [SEP] DOC2",1,0,"Dab, mitogen-responsive phosphoprotein, homolog 2 (Drosophila)",DOC2,False,A31834279,1,Genes & Molecular Sequences,DAB2 gene
172,Streptotrix mycetomatis [SEP] Madurella ikedae,1,0,Streptotrix mycetomatis,Madurella ikedae,False,A17543024,3,Living Beings,Madurella mycetomatis
178,Theg homolog (mouse) [SEP] theg spermatid protein,1,0,Theg homolog (mouse),theg spermatid protein,False,A31852570,1,Genes & Molecular Sequences,THEG gene
404,"BENZALKONIUM CHLORIDE 0.001 TOPICAL CLOTH [Germ Away] [SEP] benzalkonium chloride 0.1 % TOPICAL ADHESIVE PATCH, MEDICATED",1,0,BENZALKONIUM CHLORIDE 0.001 TOPICAL CLOTH [Germ Away],"benzalkonium chloride 0.1 % TOPICAL ADHESIVE PATCH, MEDICATED",False,A32088035,1,Chemicals & Drugs,benzalkonium chloride 1 MG/ML Medicated Pad
467,Leucocarbo pelagicus [SEP] Phalacrocorax pelagicus (organism),1,0,Leucocarbo pelagicus,Phalacrocorax pelagicus (organism),False,A31911286,1,Living Beings,Phalacrocorax pelagicus
653,"BENZALKONIUM CHLORIDE 1 mg in 1 g TOPICAL CLOTH [al-SHIELDS Disposable Antibacterial Hand Wipes with Clean Lemon Scent] [SEP] benzalkonium chloride 0.1 % TOPICAL ADHESIVE PATCH, MEDICATED",1,0,BENZALKONIUM CHLORIDE 1 mg in 1 g TOPICAL CLOTH [al-SHIELDS Disposable Antibacterial Hand Wipes with Clean Lemon Scent],"benzalkonium chloride 0.1 % TOPICAL ADHESIVE PATCH, MEDICATED",False,A32335988,1,Chemicals & Drugs,benzalkonium chloride 1 MG/ML Medicated Pad
671,"SORAVTANSINE [SEP] DM4, Maytansinoid",1,0,SORAVTANSINE,"DM4, Maytansinoid",False,A31602465,1,Chemicals & Drugs,Maytansinoid DM4
784,"Serum Free Immunoglobulin Light Chain, Lambda [SEP] Lambda Light Chain, Free",1,0,"Serum Free Immunoglobulin Light Chain, Lambda","Lambda Light Chain, Free",False,A31607500,1,Procedures,Free Lambda Light Chain Measurement
938,benzoylmethylecgonine [SEP] Cocaine,1,0,benzoylmethylecgonine,Cocaine,False,A32335305,1,Chemicals & Drugs,cocaine
951,H2A.Z.1 [SEP] H2AZ1,1,0,H2A.Z.1,H2AZ1,False,A31608377,2,Chemicals & Drugs,Histone H2A.Z


In [40]:
#False Positives
fps = df[(df['label'] != df['pred']) & (df['label'] == 0)]

print(len(fps))
fps[['queries','candidates','label','pred']][:100]

15778


,queries,candidates,label,pred
22,arginine/serine-rich protein 1,SRA1,0,1
27,"ATP synthase, H+ transporting, mitochondrial Fo complex, subunit F2 pseudogene 3",ATP5MC3 gene,0,1
45,SDYS,SDYS,0,1
54,BENZALKONIUM CHLORIDE 1 mg in 1 g TOPICAL CLOTH [Antiseptic Towel Benzalkonium Chloride],Pro Pet Dental Wipes 0.1 % Medicated Pad,0,1
55,transfer RNA tyrosine 1 (anticodon GUA),TRT-AGT2-1 gene,0,1
57,WW domain binding protein 1-like pseudogene 4,WBP4 gene,0,1
65,BENZALKONIUM CHLORIDE 0.1 g in 100 g TOPICAL CLOTH [Sanitizing Wipes],BENZALKONIUM CHLORIDE 0.115 g TOPICAL CLOTH [Hand Wipes antibacterial],0,1
77,BENZALKONIUM CHLORIDE 0.1 g in 100 g TOPICAL LIQUID [pullio wipes for faith],BENZALKONIUM CHLORIDE 0.115 g TOPICAL CLOTH [Equate Antibacterial Citrus Hand Wipes],0,1
81,protirelin,"Thyrotropin-releasing factor, prepro-",0,1
101,sodium hyaluronate 23 MG/ML Injectable Solution,HYALURONATE NA (GEL-ONE) 10MG/ML SYR 3ML,0,1


In [38]:
for i,g in fps.groupby('sem_group'):
    display(g.sort_values('queries').sample(10,replace=True,random_state=np.random.RandomState(42)))

,syn_pair,label,pred,queries,candidates,correct,query_auis,num_auis,sem_group,preferred_name
59700,Adjustment [SEP] adapting,0,1,Adjustment,adapting,False,A6917088,8,Activities & Behaviors,Individual Adjustment
141655,Adjustment [SEP] Adapt,0,1,Adjustment,Adapt,False,A6917088,8,Activities & Behaviors,Individual Adjustment
43671,Ultramarathon Running [SEP] Running or jogging 1/2 mile,0,1,Ultramarathon Running,Running or jogging 1/2 mile,False,A32280833,1,Activities & Behaviors,Ultramarathon Running
113107,Adjustment [SEP] adjusted,0,1,Adjustment,adjusted,False,A6917088,8,Activities & Behaviors,Individual Adjustment
129249,Adjustment [SEP] ADAPT,0,1,Adjustment,ADAPT,False,A6917088,8,Activities & Behaviors,Individual Adjustment
70221,Adjustment [SEP] adjustment reaction,0,1,Adjustment,adjustment reaction,False,A6917088,8,Activities & Behaviors,Individual Adjustment
122266,"Adjustment [SEP] Adaptation, Psychological",0,1,Adjustment,"Adaptation, Psychological",False,A6917088,8,Activities & Behaviors,Individual Adjustment
60526,Adjustment [SEP] ADAPT,0,1,Adjustment,ADAPT,False,A6917088,8,Activities & Behaviors,Individual Adjustment
69616,Adjustment [SEP] Adaptic,0,1,Adjustment,Adaptic,False,A6917088,8,Activities & Behaviors,Individual Adjustment
100726,"Adjustment [SEP] Adaptation, function (observable entity)",0,1,Adjustment,"Adaptation, function (observable entity)",False,A6917088,8,Activities & Behaviors,Individual Adjustment


,syn_pair,label,pred,queries,candidates,correct,query_auis,num_auis,sem_group,preferred_name
36775,Esophageal; Proximal [SEP] T11 part of esophagus,0,1,Esophageal; Proximal,T11 part of esophagus,False,A31597502,1,Anatomy,Upper third of esophagus
135860,"sacral part of spinal cord [SEP] SPINAL CORD, SACRAL",0,1,sacral part of spinal cord,"SPINAL CORD, SACRAL",False,A32071422,1,Anatomy,Segment of sacral spinal cord
39070,Structure of anterior medullary spinocerebellar tract [SEP] anterior spinothalamic tract of the medulla,0,1,Structure of anterior medullary spinocerebellar tract,anterior spinothalamic tract of the medulla,False,A32318007,1,Anatomy,Structure of anterior medullary spinocerebellar tract
12304,Paneth Cell [SEP] enterocyte,0,1,Paneth Cell,enterocyte,False,A7669163,2,Anatomy,Paneth Cells
120636,Esophageal; Proximal [SEP] T10 part of esophagus,0,1,Esophageal; Proximal,T10 part of esophagus,False,A31597502,1,Anatomy,Upper third of esophagus
15344,"Cell, Paneth [SEP] Gastrointestinal Endocrine Cell",0,1,"Cell, Paneth",Gastrointestinal Endocrine Cell,False,A32283032,1,Anatomy,Paneth Cells
110655,Medulla oblongata ventral spinocerebellar tract [SEP] ventral crossed tract,0,1,Medulla oblongata ventral spinocerebellar tract,ventral crossed tract,False,A32315497,2,Anatomy,Structure of anterior medullary spinocerebellar tract
119976,Anterior medullary spinocerebellar tract [SEP] Tractus spinocerebellaris anterior,0,1,Anterior medullary spinocerebellar tract,Tractus spinocerebellaris anterior,False,A32312908,1,Anatomy,Structure of anterior medullary spinocerebellar tract
36775,Esophageal; Proximal [SEP] T11 part of esophagus,0,1,Esophageal; Proximal,T11 part of esophagus,False,A31597502,1,Anatomy,Upper third of esophagus
79808,MAIT Cell [SEP] Tumor-Infiltrating Immune Cell,0,1,MAIT Cell,Tumor-Infiltrating Immune Cell,False,A32292944,1,Anatomy,Mucosal-Associated Invariant T-Cell


,syn_pair,label,pred,queries,candidates,correct,query_auis,num_auis,sem_group,preferred_name
35782,infliximab-axxq 100 mg INTRAVEN VIAL (EA) [SEP] Product containing precisely infliximab 100 milligram/1 vial powder for conventional release solution for injection (clinical drug),0,1,infliximab-axxq 100 mg INTRAVEN VIAL (EA),Product containing precisely infliximab 100 milligram/1 vial powder for conventional release solution for injection (clinical drug),False,A31520132,2,Chemicals & Drugs,infliximab-axxq 100 MG Injection
105003,"sodium hyaluronate 23 MG/ML Injectable Solution [SEP] HYALURONATE NA 30MG/ML INJ,OPH,SYRINGE,0.65ML",0,1,sodium hyaluronate 23 MG/ML Injectable Solution,"HYALURONATE NA 30MG/ML INJ,OPH,SYRINGE,0.65ML",False,A31690727,1,Chemicals & Drugs,sodium hyaluronate 23 MG/ML Injectable Solution
26280,BENZALKONIUM CHLORIDE 0.0004 g in 0.4 g TOPICAL SWAB [Olje Antibacterial Wipes] [SEP] BENZALKONIUM CHLORIDE 0.13 TOPICAL SWAB [Wet Ones Antibacterial Hand Wipes Tropical Splash],0,1,BENZALKONIUM CHLORIDE 0.0004 g in 0.4 g TOPICAL SWAB [Olje Antibacterial Wipes],BENZALKONIUM CHLORIDE 0.13 TOPICAL SWAB [Wet Ones Antibacterial Hand Wipes Tropical Splash],False,A31796574,1,Chemicals & Drugs,benzalkonium chloride 1 MG/ML Medicated Pad
137005,Equaline Lecithin 1200mg Softgel [SEP] Walgreens Finest Natural Lecithin 1200mg Softgel,0,1,Equaline Lecithin 1200mg Softgel,Walgreens Finest Natural Lecithin 1200mg Softgel,False,A32332861,1,Chemicals & Drugs,soybean lecithin 1200 MG Oral Capsule
123269,CPT [SEP] CPT 11,0,1,CPT,CPT 11,False,A4360697,19,Chemicals & Drugs,captopril
115116,"CHOLECALCIF 50MCG (D3-2,000UNIT) TAB [SEP] vitamin D3 10 MCG (400 UNT) Oral Tablet [Delta D3]",0,1,"CHOLECALCIF 50MCG (D3-2,000UNIT) TAB",vitamin D3 10 MCG (400 UNT) Oral Tablet [Delta D3],False,A31624863,1,Chemicals & Drugs,cholecalciferol 0.05 MG Oral Tablet
60574,BENZALKONIUM CHLORIDE 0.10 g in 100 g TOPICAL SWAB [Premium Nature Antibacterial Wipes] [SEP] BENZALKONIUM CHLORIDE 0.115 g TOPICAL CLOTH [Simply Done],0,1,BENZALKONIUM CHLORIDE 0.10 g in 100 g TOPICAL SWAB [Premium Nature Antibacterial Wipes],BENZALKONIUM CHLORIDE 0.115 g TOPICAL CLOTH [Simply Done],False,A32090748,2,Chemicals & Drugs,benzalkonium chloride 1 MG/ML Medicated Pad
112731,BENZALKONIUM CHLORIDE 0.1 g in 100 mL TOPICAL CLOTH [Travel Safe Hand Sanitizer] [SEP] BENZALKONIUM CHLORIDE 0.1 g in 100 mL TOPICAL SOLUTION [Puracy Natural Foaming Hand Sanitizer],0,1,BENZALKONIUM CHLORIDE 0.1 g in 100 mL TOPICAL CLOTH [Travel Safe Hand Sanitizer],BENZALKONIUM CHLORIDE 0.1 g in 100 mL TOPICAL SOLUTION [Puracy Natural Foaming Hand Sanitizer],False,A32333253,1,Chemicals & Drugs,benzalkonium chloride 1 MG/ML Medicated Pad
6275,"Lactamase Inhibitor, beta [SEP] (INACTIVE) BETA-LACTAM ANTIMICROBIALS",0,1,"Lactamase Inhibitor, beta",(INACTIVE) BETA-LACTAM ANTIMICROBIALS,False,A32292343,1,Chemicals & Drugs,beta-Lactamase Inhibitors
70739,Today's Health Soya Lecithin 1200mg Softgel [SEP] LECITHIN 1200MG CAP,0,1,Today's Health Soya Lecithin 1200mg Softgel,LECITHIN 1200MG CAP,False,A32337684,1,Chemicals & Drugs,soybean lecithin 1200 MG Oral Capsule


,syn_pair,label,pred,queries,candidates,correct,query_auis,num_auis,sem_group,preferred_name
140181,"Phylogenetic Inferences [SEP] DNA Barcodings, Phylogenetic",0,1,Phylogenetic Inferences,"DNA Barcodings, Phylogenetic",False,A32289268,1,Concepts & Ideas,Phylogenetic Inference
47684,WPAI0201 [SEP] WPAI0103,0,1,WPAI0201,WPAI0103,False,A31598754,1,Concepts & Ideas,WPAI-GH Version 2 - Currently Employed
99605,Phylogenetic Inferences [SEP] phylogeny,0,1,Phylogenetic Inferences,phylogeny,False,A32289268,1,Concepts & Ideas,Phylogenetic Inference
30579,Hemizygous [SEP] Hemizygote,0,1,Hemizygous,Hemizygote,False,A24097807,3,Concepts & Ideas,Hemizygosity
78776,Private prescription &#x7C; Glasses lens prescription &#x7C; Eyeglasses [SEP] Private prescription:Arb:Pt:Glasses prescription:Ord,0,1,Private prescription &#x7C; Glasses lens prescription &#x7C; Eyeglasses,Private prescription:Arb:Pt:Glasses prescription:Ord,False,A32078743,1,Concepts & Ideas,Private prescription &#x7C; Glasses lens prescription &#x7C; Eyeglasses
116536,"Phylogenetic Inference [SEP] Barcode, Phylogenetic DNA",0,1,Phylogenetic Inference,"Barcode, Phylogenetic DNA",False,A32287818,1,Concepts & Ideas,Phylogenetic Inference
50119,identifier [SEP] ID,0,1,identifier,ID,False,A18622480,3,Concepts & Ideas,Identifier
81633,Hemizygous [SEP] hemi,0,1,Hemizygous,hemi,False,A24097807,3,Concepts & Ideas,Hemizygosity
140181,"Phylogenetic Inferences [SEP] DNA Barcodings, Phylogenetic",0,1,Phylogenetic Inferences,"DNA Barcodings, Phylogenetic",False,A32289268,1,Concepts & Ideas,Phylogenetic Inference
48257,Quality Adjusted Life Year [SEP] per years,0,1,Quality Adjusted Life Year,per years,False,A32292254,1,Concepts & Ideas,Quality-Adjusted Life Years


,syn_pair,label,pred,queries,candidates,correct,query_auis,num_auis,sem_group,preferred_name
92625,"upper extremity prosthetic addition, rotation wrist unit with cable lock (treatment) [SEP] Upper extremity addition, rotation wrist unit with cable lock",0,1,"upper extremity prosthetic addition, rotation wrist unit with cable lock (treatment)","Upper extremity addition, rotation wrist unit with cable lock",False,A32265218,1,Devices,"upper extremity prosthetic addition, rotation wrist unit with cable lock"
53437,{24 (ethinyl estradiol 0.02 MG / norethindrone acetate 1 MG Oral Tablet) / 4 (ferrous fumarate 75 MG Oral Tablet) } Pack [Larin 24 Fe 1/20] [SEP] Ethinyl Estradiol/Norethindrone Acetate;Ferrous Fumarate 20 MCG-1 MG; 75 MG Oral Tablet [LARIN FE 1/20],0,1,{24 (ethinyl estradiol 0.02 MG / norethindrone acetate 1 MG Oral Tablet) / 4 (ferrous fumarate 75 MG Oral Tablet) } Pack [Larin 24 Fe 1/20],Ethinyl Estradiol/Norethindrone Acetate;Ferrous Fumarate 20 MCG-1 MG; 75 MG Oral Tablet [LARIN FE 1/20],False,A31686081,1,Devices,{24 (Ethinyl Estradiol 0.02 MG / norethindrone acetate 1 MG Oral Tablet) / 4 (Ferrous fumarate 75 MG Oral Tablet) } Pack [Larin 24 Fe 1/20]
59813,"upper extremity prosthetic addition, rotation wrist unit with cable lock [SEP] Upper extremity addition, rotation wrist unit with cable lock",0,1,"upper extremity prosthetic addition, rotation wrist unit with cable lock","Upper extremity addition, rotation wrist unit with cable lock",False,A32268739,1,Devices,"upper extremity prosthetic addition, rotation wrist unit with cable lock"
53437,{24 (ethinyl estradiol 0.02 MG / norethindrone acetate 1 MG Oral Tablet) / 4 (ferrous fumarate 75 MG Oral Tablet) } Pack [Larin 24 Fe 1/20] [SEP] Ethinyl Estradiol/Norethindrone Acetate;Ferrous Fumarate 20 MCG-1 MG; 75 MG Oral Tablet [LARIN FE 1/20],0,1,{24 (ethinyl estradiol 0.02 MG / norethindrone acetate 1 MG Oral Tablet) / 4 (ferrous fumarate 75 MG Oral Tablet) } Pack [Larin 24 Fe 1/20],Ethinyl Estradiol/Norethindrone Acetate;Ferrous Fumarate 20 MCG-1 MG; 75 MG Oral Tablet [LARIN FE 1/20],False,A31686081,1,Devices,{24 (Ethinyl Estradiol 0.02 MG / norethindrone acetate 1 MG Oral Tablet) / 4 (Ferrous fumarate 75 MG Oral Tablet) } Pack [Larin 24 Fe 1/20]
53437,{24 (ethinyl estradiol 0.02 MG / norethindrone acetate 1 MG Oral Tablet) / 4 (ferrous fumarate 75 MG Oral Tablet) } Pack [Larin 24 Fe 1/20] [SEP] Ethinyl Estradiol/Norethindrone Acetate;Ferrous Fumarate 20 MCG-1 MG; 75 MG Oral Tablet [LARIN FE 1/20],0,1,{24 (ethinyl estradiol 0.02 MG / norethindrone acetate 1 MG Oral Tablet) / 4 (ferrous fumarate 75 MG Oral Tablet) } Pack [Larin 24 Fe 1/20],Ethinyl Estradiol/Norethindrone Acetate;Ferrous Fumarate 20 MCG-1 MG; 75 MG Oral Tablet [LARIN FE 1/20],False,A31686081,1,Devices,{24 (Ethinyl Estradiol 0.02 MG / norethindrone acetate 1 MG Oral Tablet) / 4 (Ferrous fumarate 75 MG Oral Tablet) } Pack [Larin 24 Fe 1/20]
50491,EasyPoint 20G 1 inch Needle [SEP] NEEDLE 20G 1IN,0,1,EasyPoint 20G 1 inch Needle,NEEDLE 20G 1IN,False,A31798420,1,Devices,EasyPoint 20G 1 inch Needle
59813,"upper extremity prosthetic addition, rotation wrist unit with cable lock [SEP] Upper extremity addition, rotation wrist unit with cable lock",0,1,"upper extremity prosthetic addition, rotation wrist unit with cable lock","Upper extremity addition, rotation wrist unit with cable lock",False,A32268739,1,Devices,"upper extremity prosthetic addition, rotation wrist unit with cable lock"
59813,"upper extremity prosthetic addition, rotation wrist unit with cable lock [SEP] Upper extremity addition, rotation wrist unit with cable lock",0,1,"upper extremity prosthetic addition, rotation wrist unit with cable lock","Upper extremity addition, rotation wrist unit with cable lock",False,A32268739,1,Devices,"upper extremity prosthetic addition, rotation wrist unit with cable lock"
59813,"upper extremity prosthetic addition, rotation wrist unit with cable lock [SEP] Upper extremity addition, rotation wrist unit with cable lo

...

,syn_pair,label,pred,queries,candidates,correct,query_auis,num_auis,sem_group,preferred_name
76132,Campylobacter fetus subsp. testudinum [SEP] Campylobacter BCW_6878,0,1,Campylobacter fetus subsp. testudinum,Campylobacter BCW_6878,False,A32308191,2,Living Beings,Campylobacter fetus subsp. testudinum
65683,Human herpesvirus 6 type B [SEP] human herpesvirus 6,0,1,Human herpesvirus 6 type B,human herpesvirus 6,False,A31963171,1,Living Beings,"Herpesvirus 6B, Human"
127693,Streptotrix mycetomatis [SEP] mycetoma,0,1,Streptotrix mycetomatis,mycetoma,False,A17543024,3,Living Beings,Madurella mycetomatis
35112,Candida albicans var. tropicalis [SEP] CANDIDA PARAPSILOSIS,0,1,Candida albicans var. tropicalis,CANDIDA PARAPSILOSIS,False,A23753830,2,Living Beings,Candida tropicalis
29360,Campylobacter fetus subsp. testudinum [SEP] CAMPYLOBACTER JEJUNI,0,1,Campylobacter fetus subsp. testudinum,CAMPYLOBACTER JEJUNI,False,A32308191,2,Living Beings,Campylobacter fetus subsp. testudinum
32314,Bacteriophage h1 [SEP] H1-1,0,1,Bacteriophage h1,H1-1,False,A31938632,1,Living Beings,Bacillus phage H1
87098,Olea hensoni [SEP] Burbot liver oil (substance),0,1,Olea hensoni,Burbot liver oil (substance),False,A31941553,1,Living Beings,Olea hensoni
11278,Acacia arabica [SEP] Acacia,0,1,Acacia arabica,Acacia,False,A27672097,6,Living Beings,Acacia
131241,Majumderatax [SEP] Calendula,0,1,Majumderatax,Calendula,False,A20469349,1,Living Beings,Majumderatax
47374,Campylobacter fetus subsp. testudinum [SEP] Campylobacter jejuni ss doylei,0,1,Campylobacter fetus subsp. testudinum,Campylobacter jejuni ss doylei,False,A32308191,2,Living Beings,Campylobacter fetus subsp. testudinum


,syn_pair,label,pred,queries,candidates,correct,query_auis,num_auis,sem_group,preferred_name
121191,coconut oil [SEP] Coconut (Food),0,1,coconut oil,Coconut (Food),False,A18594942,5,Objects,coconut oil
65828,coconut oil [SEP] COCONUT,0,1,coconut oil,COCONUT,False,A18594942,5,Objects,coconut oil
13051,coconut oil [SEP] Coconut Alcohol,0,1,coconut oil,Coconut Alcohol,False,A18594942,5,Objects,coconut oil
132672,coconut oil [SEP] Coconut,0,1,coconut oil,Coconut,False,A18594942,5,Objects,coconut oil
132231,"coconut oil [SEP] Coconut oil, fractioned",0,1,coconut oil,"Coconut oil, fractioned",False,A18594942,5,Objects,coconut oil
114948,coconut oil [SEP] Fractionated coconut oil,0,1,coconut oil,Fractionated coconut oil,False,A18594942,5,Objects,coconut oil
13051,coconut oil [SEP] Coconut Alcohol,0,1,coconut oil,Coconut Alcohol,False,A18594942,5,Objects,coconut oil
60751,coconut oil [SEP] coconut,0,1,coconut oil,coconut,False,A18594942,5,Objects,coconut oil
121191,coconut oil [SEP] Coconut (Food),0,1,coconut oil,Coconut (Food),False,A18594942,5,Objects,coconut oil
43148,coconut oil [SEP] Coconut,0,1,coconut oil,Coconut,False,A18594942,5,Objects,coconut oil


,syn_pair,label,pred,queries,candidates,correct,query_auis,num_auis,sem_group,preferred_name
125461,Radiowaves [SEP] Radiant,0,1,Radiowaves,Radiant,False,A32284905,1,Phenomena,Radio Waves
137800,Radiowaves [SEP] microwave radiation,0,1,Radiowaves,microwave radiation,False,A32284905,1,Phenomena,Radio Waves
82925,Radiowave [SEP] Radiation (physical force),0,1,Radiowave,Radiation (physical force),False,A32293666,1,Phenomena,Radio Waves
57418,Radiowaves [SEP] electromagnetic radiations,0,1,Radiowaves,electromagnetic radiations,False,A32284905,1,Phenomena,Radio Waves
56213,Radiowave [SEP] radiation,0,1,Radiowave,radiation,False,A32293666,1,Phenomena,Radio Waves
65777,Radiowave [SEP] Grenz radiation,0,1,Radiowave,Grenz radiation,False,A32293666,1,Phenomena,Radio Waves
125461,Radiowaves [SEP] Radiant,0,1,Radiowaves,Radiant,False,A32284905,1,Phenomena,Radio Waves
62531,Radiowaves [SEP] light waves,0,1,Radiowaves,light waves,False,A32284905,1,Phenomena,Radio Waves
46121,Radiowave [SEP] Microwave Radiation,0,1,Radiowave,Microwave Radiation,False,A32293666,1,Phenomena,Radio Waves
142820,Radiowaves [SEP] Microwave Radiation,0,1,Radiowaves,Microwave Radiation,False,A32284905,1,Phenomena,Radio Waves


,syn_pair,label,pred,queries,candidates,correct,query_auis,num_auis,sem_group,preferred_name
56586,Left eye KW impression code [SEP] KW impression code:Imp:Pt:Eye.right:Nom,0,1,Left eye KW impression code,KW impression code:Imp:Pt:Eye.right:Nom,False,A32074861,1,Physiology,KW impression code:Imp:Pt:Eye.left:Nom
55206,avirulence protein [SEP] AVRP,0,1,avirulence protein,AVRP,False,A32071010,1,Physiology,avirulence protein
20298,L pupil Pupillary resp [SEP] Left pupil,0,1,L pupil Pupillary resp,Left pupil,False,A32077748,1,Physiology,Pupillary response:Find:Pt:Pupil.left:Nom
4480,GD1b Gangl IgG Ser Ql IA [SEP] GD1b Gangl IgG Ser IA-aCnc,0,1,GD1b Gangl IgG Ser Ql IA,GD1b Gangl IgG Ser IA-aCnc,False,A32081086,1,Physiology,Ganglioside GD1b Ab.IgG:PrThr:Pt:Ser:Ord:IA
52275,Left pupil Pupillary response [SEP] LR - Pupillary light reaction,0,1,Left pupil Pupillary response,LR - Pupillary light reaction,False,A32077747,1,Physiology,Pupillary response:Find:Pt:Pupil.left:Nom
43374,Innovativeness [SEP] curiosity,0,1,Innovativeness,curiosity,False,A7565406,2,Physiology,Innovativeness
63350,negative regulation of interleukin-25 secretion [SEP] inhibition of interleukin-25 production,0,1,negative regulation of interleukin-25 secretion,inhibition of interleukin-25 production,False,A32069264,1,Physiology,negative regulation of interleukin-25 secretion
107356,HAdV Ag Upper resp Ql IA [SEP] Adenovirus Ag IA Ql (U),0,1,HAdV Ag Upper resp Ql IA,Adenovirus Ag IA Ql (U),False,A32073575,1,Physiology,Adenovirus Ag:PrThr:Pt:Respiratory.upper:Ord:IA
56586,Left eye KW impression code [SEP] KW impression code:Imp:Pt:Eye.right:Nom,0,1,Left eye KW impression code,KW impression code:Imp:Pt:Eye.right:Nom,False,A32074861,1,Physiology,KW impression code:Imp:Pt:Eye.left:Nom
97184,R eye Sphere--W cyclo Subj refraction [SEP] Cylinder^WO cycloplegia:Inverse Length:Point in time:Eye.right:Quantitative:Subjective refraction,0,1,R eye Sphere--W cyclo Subj refraction,Cylinder^WO cycloplegia:Inverse Length:Point in time:Eye.right:Quantitative:Subjective refraction,False,A32080582,1,Physiology,Sphere^W cycloplegia:InvLen:Pt:Eye.right:Qn:Subjective refraction


,syn_pair,label,pred,queries,candidates,correct,query_auis,num_auis,sem_group,preferred_name
127098,Excision of lesion of ampulla of Vater [SEP] Excision of ampulla of Vater,0,1,Excision of lesion of ampulla of Vater,Excision of ampulla of Vater,False,A32269877,2,Procedures,Excision of lesion of ampulla of Vater
42404,partial colectomy with anastomosis (treatment) [SEP] Partial colectomy with coloproctostomy,0,1,partial colectomy with anastomosis (treatment),Partial colectomy with coloproctostomy,False,A32260551,1,Procedures,Partial colectomy with anastomosis
12916,"Serum Free Immunoglobulin Light Chain, Lambda [SEP] Serum Free Immunoglobulin Light Chain",0,1,"Serum Free Immunoglobulin Light Chain, Lambda",Serum Free Immunoglobulin Light Chain,False,A31607500,1,Procedures,Free Lambda Light Chain Measurement
55291,Excision of lesion of ampulla of Vater [SEP] Excision of ampulla of Vater,0,1,Excision of lesion of ampulla of Vater,Excision of ampulla of Vater,False,A32269877,2,Procedures,Excision of lesion of ampulla of Vater
77762,BrdU Cellular Proliferation Assay [SEP] BrdUTP,0,1,BrdU Cellular Proliferation Assay,BrdUTP,False,A31600321,1,Procedures,BrdU Cellular Proliferation Assay
37546,"RBC antigen genotyping MNS blood group [SEP] Systems, MNSs Blood-Group",0,1,RBC antigen genotyping MNS blood group,"Systems, MNSs Blood-Group",False,A32272107,1,Procedures,"RBC antigen typing, MNS blood group"
85310,BRDU CELLULAR PROLIFERATION ASSAY [SEP] 5-Bromodeoxyuridine,0,1,BRDU CELLULAR PROLIFERATION ASSAY,5-Bromodeoxyuridine,False,A31612661,1,Procedures,BrdU Cellular Proliferation Assay
127098,Excision of lesion of ampulla of Vater [SEP] Excision of ampulla of Vater,0,1,Excision of lesion of ampulla of Vater,Excision of ampulla of Vater,False,A32269877,2,Procedures,Excision of lesion of ampulla of Vater
126865,Excision of lesion of ampulla of Vater (treatment) [SEP] Excision of ampulla of Vater,0,1,Excision of lesion of ampulla of Vater (treatment),Excision of ampulla of Vater,False,A32252284,1,Procedures,Excision of lesion of ampulla of Vater
123569,"RBC antigen typing, MNS blood group [SEP] System, MNSs Blood-Group",0,1,"RBC antigen typing, MNS blood group","System, MNSs Blood-Group",False,A32269196,1,Procedures,"RBC antigen typing, MNS blood group"


In [31]:
len(umls.aui2scui)

15229292

In [32]:
scui_df = []

for k,v in umls.scui2auis.items():
    scui_df.append((k,len(v)))
    
scui_df = pd.DataFrame(scui_df)

In [37]:
scui_df['source'] = [scui.split('|||')[1] for scui in scui_df[0]]
scui_df['scui_no_source'] = [scui.split('|||')[0] for scui in scui_df[0]]

In [81]:
# Percentage of AUIS with real source CUIs (source synonymy info)
scui_df[scui_df['scui_no_source'] != ''][1].sum()/scui_df[1].sum()

0.8177630090639909

In [82]:
auis_with_scuis_df = scui_df[scui_df['scui_no_source'] != '']

In [90]:
100*auis_with_scuis_df.groupby(1).count()/len(auis_with_scuis_df)

,0,source,scui_no_source
1,,,
1,63.009194,63.009194,63.009194
2,19.584014,19.584014,19.584014
3,9.667312,9.667312,9.667312
4,3.511689,3.511689,3.511689
5,2.470534,2.470534,2.470534
6,0.598477,0.598477,0.598477
7,0.360043,0.360043,0.360043
8,0.243005,0.243005,0.243005
9,0.152335,0.152335,0.152335


In [45]:
df['scui'] = [umls.aui2scui[aui] for aui in df.query_auis]

In [74]:
source_scui_df = []

for i,g in tqdm(scui_df.groupby('source')):
    
    no_scui = g[g.scui_no_source == ''][1].values.sum()
    source_aui_num = g[1].values.sum()
    
    source_scui_df.append((i,no_scui/source_aui_num,source_aui_num))

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 183.44it/s]


In [75]:
source_scui_df = pd.DataFrame(source_scui_df).sort_values(2,ascending=False)

In [76]:
source_scui_df.groupby(1).sum()/source_scui_df[2].values.sum()

,2
1,
0.000000,0.609139
0.000001,0.105633
0.000029,0.101452
0.000075,0.001489
0.904762,0.000561
1.000000,0.181727


In [48]:
umls.scui2auis['|||HPO']

['A30925085',
 'A24680170',
 'A24677906',
 'A24682424',
 'A27414126',
 'A27414159',
 'A24682425',
 'A28253524',
 'A28681773',
 'A24670862',
 'A28254609',
 'A28253500',
 'A28254234',
 'A24667044',
 'A30926492',
 'A24669038',
 'A28256146',
 'A26513484',
 'A24669916',
 'A24669915',
 'A30924786',
 'A24677171',
 'A24677445',
 'A27332497',
 'A29414127',
 'A30926538',
 'A24673518',
 'A24675759',
 'A24677634',
 'A24675571',
 'A24677869',
 'A24682168',
 'A30926340',
 'A24678237',
 'A30926191',
 'A24674921',
 'A27414006',
 'A24679713',
 'A24682120',
 'A24670802',
 'A27333413',
 'A29930632',
 'A29930439',
 'A24682993',
 'A24680729',
 'A28681449',
 'A24683947',
 'A24677146',
 'A24675135',
 'A24668297',
 'A24675139',
 'A24683942',
 'A24677141',
 'A24678495',
 'A24673526',
 'A24678067',
 'A24681886',
 'A28253467',
 'A24675108',
 'A24668266',
 'A28681149',
 'A24665969',
 'A24678844',
 'A24681894',
 'A27328832',
 'A24681357',
 'A28256337',
 'A28256338',
 'A24667761',
 'A24679738',
 'A28255695',
 'A246

In [ ]:
full_dev_set = pd.read_csv('/data/Bodenreider_UMLS_DL/Interns/Bernal/rw_uva/output/2/classifiers/0/one-way.train.tsv',sep='\t')
full_dev_set.groupby('labels').count()/len(full_dev_set)

In [ ]:
full_dev_set.groupby('labels').count()

In [ ]:
full_dev_set = pd.read_csv('/data/Bodenreider_UMLS_DL/Interns/Bernal/rw_uva/output/2/classifiers/0/one-way.dev.tsv',sep='\t')
full_dev_set.groupby('labels').count()/len(full_dev_set)

In [ ]:
full_dev_set = pd.read_csv('/data/Bodenreider_UMLS_DL/Interns/Bernal/rw_uva/output/2/classifiers/0/one-way.test.tsv',sep='\t')
full_dev_set.groupby('labels').count()/len(full_dev_set)

In [ ]:
full_dev_set = pd.read_csv('/data/Bodenreider_UMLS_DL/Interns/Bernal/rw_uva/output/2/classifiers/1/one-way.dev.tsv',sep='\t')
full_dev_set.groupby('labels').count()/len(full_dev_set)

In [ ]:
full_dev_set = pd.read_csv('/data/Bodenreider_UMLS_DL/Interns/Bernal/rw_uva/output/2/classifiers/1/one-way.test.tsv',sep='\t')
full_dev_set.groupby('labels').count()/len(full_dev_set)